<a href="https://colab.research.google.com/github/Ash-Stat/An-Si-Hyun/blob/data/kaggle_titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EDA

In [428]:
import pandas as pd
data=pd.read_csv('./train.csv')
data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [429]:
# find a NaN
data.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [430]:
# drop featureless columns
data=data.drop(['PassengerId','Name'],axis=1)
data

,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,3,male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,1,female,38.0,1,0,PC 17599,71.2833,C85,C
2,1,3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,1,female,35.0,1,0,113803,53.1000,C123,S
4,0,3,male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,211536,13.0000,NaN,S
887,1,1,female,19.0,0,0,112053,30.0000,B42,S
888,0,3,female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,1,1,male,26.0,0,0,111369,30.0000,C148,C


In [431]:
# find unique values
# 잘못 입력된 값이 있는지 확인
for i in data.columns:
  print(i,':', data[i].unique())

Survived : [0 1]
Pclass : [3 1 2]
Sex : ['male' 'female']
Age : [22.   38.   26.   35.     nan 54.    2.   27.   14.    4.   58.   20.
 39.   55.   31.   34.   15.   28.    8.   19.   40.   66.   42.   21.
 18.    3.    7.   49.   29.   65.   28.5   5.   11.   45.   17.   32.
 16.   25.    0.83 30.   33.   23.   24.   46.   59.   71.   37.   47.
 14.5  70.5  32.5  12.    9.   36.5  51.   55.5  40.5  44.    1.   61.
 56.   50.   36.   45.5  20.5  62.   41.   52.   63.   23.5   0.92 43.
 60.   10.   64.   13.   48.    0.75 53.   57.   80.   70.   24.5   6.
  0.67 30.5   0.42 34.5  74.  ]
SibSp : [1 0 3 4 2 5 8]
Parch : [0 1 2 5 3 4 6]
Ticket : ['A/5 21171' 'PC 17599' 'STON/O2. 3101282' '113803' '373450' '330877'
 '17463' '349909' '347742' '237736' 'PP 9549' '113783' 'A/5. 2151'
 '347082' '350406' '248706' '382652' '244373' '345763' '2649' '239865'
 '248698' '330923' '113788' '347077' '2631' '19950' '330959' '349216'
 'PC 17601' 'PC 17569' '335677' 'C.A. 24579' 'PC 17604' '113789' '2677'


In [432]:
# 칼럼별 생존과의 연관성을 확인 : 오즈비
# 데이터셋 준비
import numpy as np
odds=pd.crosstab(index=data.Pclass, columns=data.Survived,values=data.Survived, aggfunc=pd.value_counts )
odds.loc[1]=odds.loc[1]/216
odds.loc[2]=odds.loc[2]/184
odds.loc[3]=odds.loc[3]/491
odds=odds[[1,0]]
odds

Survived,1,0
Pclass,,
1,0.629630,0.370370
2,0.472826,0.527174
3,0.242363,0.757637


In [433]:
#Pclass=1 vs 2
print('odds of Pclass 1 vs 2 :', 1/((80*87)/(136*97)))
print('odds of Pclass 2 vs 3 :', 1/((97*119)/(87*372)))
print('odds of Pclass 1 vs 3 :', 1/((80*119)/(136*372)))

odds of Pclass 1 vs 2 : 1.8954022988505748
odds of Pclass 2 vs 3 : 2.803777180975483
odds of Pclass 1 vs 3 : 5.314285714285714


# Pclass2가 1보다 생존비율이 높다
# Pclass3이 2보다 생존비율이 높다.
# Pclass3이 1보다 생존비율이 월등히 높다.

In [434]:
# 아노바 분석
chi_data=pd.crosstab(index=data.Pclass, columns=data.Survived,values=data.Survived, aggfunc=pd.value_counts )
chi_data

Survived,0,1
Pclass,,
1,80,136
2,97,87
3,372,119


In [435]:
import numpy as np
from scipy.stats import chi2_contingency
chi, p, dof, expected = chi2_contingency(chi_data)

print('Survived and Pclass')
print()
print(f"chi 스퀘어 값: {chi}",
      f"p-value (0.05): {p}",
      f"자유도 수: {dof}",
      f"기대값: \n{pd.DataFrame(expected)}",
      f"측정값: \n{chi_data}", sep = "\n" )
print('p-value가 0.05 이하 이므로, 귀무가설을 기각한다. 즉, 생존과 Pclass는 관계가 있다.')

Survived and Pclass

chi 스퀘어 값: 102.88898875696056
p-value (0.05): 4.549251711298793e-23
자유도 수: 2
기대값: 
            0           1
0  133.090909   82.909091
1  113.373737   70.626263
2  302.535354  188.464646
측정값: 
Survived    0    1
Pclass            
1          80  136
2          97   87
3         372  119
p-value가 0.05 이하 이므로, 귀무가설을 기각한다. 즉, 생존과 Pclass는 관계가 있다.


In [436]:
#성별과 생존
odds_sex_survived=pd.crosstab(index=data.Sex, columns=data.Survived,values=data.Survived, aggfunc=pd.value_counts )
odds_sex_survived=odds_sex_survived[[1,0]]
odds_sex_survived.loc['female']=odds_sex_survived.loc['female']/314
odds_sex_survived.loc['male']=odds_sex_survived.loc['male']/577
odds_sex_survived

Survived,1,0
Sex,,
female,0.742038,0.257962
male,0.188908,0.811092


In [437]:
print('odds of Sex female vs male :', (odds_sex_survived[1][0] * odds_sex_survived[0][1]) / (odds_sex_survived[1][1] * odds_sex_survived[0][0]))

odds of Sex female vs male : 12.3506625891947


# 여성이 남성보다 생존확률이 월등히 높다

In [438]:
# 아노바 분석
chi_Sex=pd.crosstab(index=data.Sex, columns=data.Survived,values=data.Survived, aggfunc=pd.value_counts )
chi_Sex=chi_Sex[[1,0]]
chi1, p1, dof1, expected1 = chi2_contingency(chi_Sex)

print('Survived and Sex')
print()
print(f"chi 스퀘어 값: {chi1}",
      f"p-value (0.05): {p1}",
      f"자유도 수: {dof1}",
      f"기대값: \n{pd.DataFrame(expected1)}",
      f"측정값: \n{chi_Sex}", sep = "\n" )
print('p-value가 0.05 이하 이므로, 귀무가설을 기각한다. 즉, 생존과 Sex는 관계가 있다.')

Survived and Sex

chi 스퀘어 값: 260.71702016732104
p-value (0.05): 1.1973570627755645e-58
자유도 수: 1
기대값: 
            0           1
0  120.525253  193.474747
1  221.474747  355.525253
측정값: 
Survived    1    0
Sex               
female    233   81
male      109  468
p-value가 0.05 이하 이므로, 귀무가설을 기각한다. 즉, 생존과 Sex는 관계가 있다.


# Age의 결측값을 어떻게 채울까??

Maximum Likelihood & 베이즈 공식 이용

In [439]:
sample=data[['Survived','Sex','Pclass','Age','SibSp','Parch']]
sample=sample.dropna()
sample

,Survived,Sex,Pclass,Age,SibSp,Parch
0,0,male,3,22.0,1,0
1,1,female,1,38.0,1,0
2,1,female,3,26.0,0,0
3,1,female,1,35.0,1,0
4,0,male,3,35.0,0,0
...,...,...,...,...,...,...
885,0,female,3,39.0,0,5
886,0,male,2,27.0,0,0
887,1,female,1,19.0,0,0
889,1,male,1,26.0,0,0


In [440]:
#Age를 나이대로 바꾼다.

for older,ind in zip(sample['Age'],sample.index):
  if older <10 :
    sample['Age'].loc[ind]=0 
  elif older <20:
    sample['Age'].loc[ind]=1
  elif older <30:
    sample['Age'].loc[ind]=2
  elif older <40:
    sample['Age'].loc[ind]=3
  elif older <50:
    sample['Age'].loc[ind]=4
  elif older <60:
    sample['Age'].loc[ind]=5
  elif older <70:
    sample['Age'].loc[ind]=6
  else:
    sample['Age'].loc[ind]=7
sample

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


,Survived,Sex,Pclass,Age,SibSp,Parch
0,0,male,3,2.0,1,0
1,1,female,1,3.0,1,0
2,1,female,3,2.0,0,0
3,1,female,1,3.0,1,0
4,0,male,3,3.0,0,0
...,...,...,...,...,...,...
885,0,female,3,3.0,0,5
886,0,male,2,2.0,0,0
887,1,female,1,1.0,0,0
889,1,male,1,2.0,0,0


In [441]:
# 일단 결측값을 채우기 전에 완전히 주어진 데이터로만 오즈비와 카이제곱 검정을 해본다.

#성별과 생존
odds_age_survived=pd.crosstab(index=sample.Age, columns=sample.Survived,values=sample.Survived, aggfunc=pd.value_counts )
odds_age_survived=odds_age_survived[[1,0]]
odds_age_survived
odds_age_survived['sum']=odds_age_survived[1]+odds_age_survived[0]
odds_age_survived[1]=odds_age_survived[1]/odds_age_survived['sum']
odds_age_survived[0]=odds_age_survived[0]/odds_age_survived['sum']
odds_age_survived=odds_age_survived.drop(['sum'],axis=1)
odds_age_survived

Survived,1,0
Age,,
0.0,0.612903,0.387097
1.0,0.401961,0.598039
2.0,0.350000,0.650000
3.0,0.437126,0.562874
4.0,0.382022,0.617978
5.0,0.416667,0.583333
6.0,0.315789,0.684211
7.0,0.142857,0.857143


In [442]:
def odds(dat,init,m):
  for n in range(init,m):
    print('odds %ds vs %ds:' % (init,n+1),(dat[1].iloc[init] * dat[0].iloc[n+1]) / (dat[1].iloc[n+1] * dat[0].iloc[init]))

In [443]:
odds(odds_age_survived,0,7)
odds(odds_age_survived,1,7)
odds(odds_age_survived,2,7)
odds(odds_age_survived,3,7)
odds(odds_age_survived,4,7)
odds(odds_age_survived,5,7)
odds(odds_age_survived,6,7)

odds 0s vs 1s: 2.355691056910569
odds 0s vs 2s: 2.940476190476191
odds 0s vs 3s: 2.038812785388128
odds 0s vs 4s: 2.5612745098039214
odds 0s vs 5s: 2.216666666666667
odds 0s vs 6s: 3.4305555555555562
odds 0s vs 7s: 9.5
odds 1s vs 2s: 1.2482435597189698
odds 1s vs 3s: 0.865483943408938
odds 1s vs 4s: 1.087270973963356
odds 1s vs 5s: 0.9409836065573771
odds 1s vs 6s: 1.4562841530054644
odds 1s vs 7s: 4.032786885245902
odds 2s vs 3s: 0.6933614330874605
odds 2s vs 4s: 0.8710407239819004
odds 2s vs 5s: 0.7538461538461537
odds 2s vs 6s: 1.1666666666666667
odds 2s vs 7s: 3.2307692307692304
odds 3s vs 4s: 1.256257822277847
odds 3s vs 5s: 1.0872340425531914
odds 3s vs 6s: 1.6826241134751774
odds 3s vs 7s: 4.659574468085107
odds 4s vs 5s: 0.8654545454545455
odds 4s vs 6s: 1.3393939393939396
odds 4s vs 7s: 3.7090909090909085
odds 5s vs 6s: 1.5476190476190477
odds 5s vs 7s: 4.2857142857142865
odds 6s vs 7s: 2.769230769230769


# 10대 이전의 생존확률은 그 어느 연령층보다 높다.

# 10대의 생존확률은 10대 이전의 생존확률을 제외하고 나머지 연련층에 비해 모두 높다.

# 20대의 생존확률은 30대를 제외하고 나머지 연령층보다 미세하게 높다.

# 30대의 생존확률은 10대와 10대 이전의 연령층 제외하고 나머지 연령층에 비해 모두 높다.

In [444]:
# 아노바 분석
chi_Age=pd.crosstab(index=sample.Age, columns=sample.Survived,values=sample.Survived, aggfunc=pd.value_counts )
chi_Age=chi_Age[[1,0]]
chi2, p2, dof2, expected2 = chi2_contingency(chi_Age)

print('Survived and Age')
print()
print(f"chi 스퀘어 값: {chi2}",
      f"p-value (0.05): {p2}",
      f"자유도 수: {dof2}",
      f"기대값: \n{pd.DataFrame(expected2)}",
      f"측정값: \n{chi_Age}", sep = "\n" )
print('p-value가 0.05 이하 이므로, 귀무가설을 기각한다. 즉, 생존과 Age는 관계가 있다.')

Survived and Age

chi 스퀘어 값: 17.42772160585894
p-value (0.05): 0.014836878112813482
자유도 수: 7
기대값: 
           0           1
0  25.182073   36.817927
1  41.428571   60.571429
2  89.355742  130.644258
3  67.829132   99.170868
4  36.148459   52.851541
5  19.495798   28.504202
6   7.717087   11.282913
7   2.843137    4.156863
측정값: 
Survived   1    0
Age              
0.0       38   24
1.0       41   61
2.0       77  143
3.0       73   94
4.0       34   55
5.0       20   28
6.0        6   13
7.0        1    6
p-value가 0.05 이하 이므로, 귀무가설을 기각한다. 즉, 생존과 Age는 관계가 있다.


# Age의 결측값을 채우기 전에 349개의 데이터에 나타나있는 Age의 평균 비율이 전체 데이터셋을 대표하는가?에 대한 검정을 시행해 본다.

# 그러기 위해선 모집단의 분포를 알아야 하지만, 중심극한정리에 의해서 모집단의 분포와 무관하게 표본평균들이 근사적으로 정규분포가 된다는 사실을 이용한다.

# 따라서 표본데이터셋이 정규분포가 된다고 가정을 하고 이에 대해서 모비율 t-test를 해본다.

# 이때 사용되는 표본표준편차 S는 모표준편차의 일치추정량이므로, 모분산이 알려져 있지 않은 경우에도 모표준편차 대신 표본표준편차를 대입하여 근사적으로 정규분포 이론을 적용할 수 있다.

# 즉, 검정 통계량은 표준정규분포를 따른다.

In [445]:
#연령대별 도수 확인
print('0s :', len(sample[sample['Age'] == 0.0]))
print('10s :', len(sample[sample['Age'] == 1.0]))
print('20s :', len(sample[sample['Age'] == 2.0]))
print('30s :', len(sample[sample['Age'] == 3.0]))
print('40s :', len(sample[sample['Age'] == 4.0]))
print('50s :', len(sample[sample['Age'] == 5.0]))
print('60s :', len(sample[sample['Age'] == 6.0]))
print('70s :', len(sample[sample['Age'] == 7.0]))

0s : 62
10s : 102
20s : 220
30s : 167
40s : 89
50s : 48
60s : 19
70s : 7


In [446]:
print('0s :', len(sample[sample['Age'] == 0.0])/714)
print('10s :', len(sample[sample['Age'] == 1.0])/714)
print('20s :', len(sample[sample['Age'] == 2.0])/714)
print('30s :', len(sample[sample['Age'] == 3.0])/714)
print('40s :', len(sample[sample['Age'] == 4.0])/714)
print('50s :', len(sample[sample['Age'] == 5.0])/714)
print('60s :', len(sample[sample['Age'] == 6.0])/714)
print('70s :', len(sample[sample['Age'] == 7.0])/714)

0s : 0.08683473389355742
10s : 0.14285714285714285
20s : 0.3081232492997199
30s : 0.2338935574229692
40s : 0.12464985994397759
50s : 0.06722689075630252
60s : 0.02661064425770308
70s : 0.00980392156862745


In [447]:
s0=len(sample[sample['Age'] == 0.0])/714
s1=len(sample[sample['Age'] == 1.0])/714
s2=len(sample[sample['Age'] == 2.0])/714
s3=len(sample[sample['Age'] == 3.0])/714
s4=len(sample[sample['Age'] == 4.0])/714
s5=len(sample[sample['Age'] == 5.0])/714
s6=len(sample[sample['Age'] == 6.0])/714
s7=len(sample[sample['Age'] == 7.0])/714

# 각 연령대별 기각역을 구하고, 그 기각역을 넘어야 귀무가설(모 데이터셋에서 연령대의 비율이 표본 데이터셋에서 구한 연령대의 비율과 같다.)을 기각할 수 있다.

60대와 70대는 표본수가 적으니 t 통계량으로 한다.

In [448]:
print('0s의 기각역 :', s0+(0.6985*np.sqrt(s0*(1-s0)/714)))
print('10s의 기각역 :', s1+(0.6985*np.sqrt(s1*(1-s1)/714)))
print('20s의 기각역 :', s2+(0.6985*np.sqrt(s2*(1-s2)/714)))
print('30s의 기각역 :', s3+(0.6985*np.sqrt(s3*(1-s3)/714)))
print('40s의 기각역 :', s4+(0.6985*np.sqrt(s4*(1-s4)/714)))
print('50s의 기각역 :', s5+(0.6985*np.sqrt(s5*(1-s5)/714)))
print('60s의 기각역 :', s6+(0.6985*np.sqrt(s6*(1-s6)/714)))
print('70s의 기각역 :', s7+(0.6985*np.sqrt(s7*(1-s7)/714)))

0s의 기각역 : 0.09419576639683828
10s의 기각역 : 0.15200448428820293
20s의 기각역 : 0.32019287793384255
30s의 기각역 : 0.2449590561238428
40s의 기각역 : 0.13328469616827715
50s의 기각역 : 0.07377290319261978
60s의 기각역 : 0.030817797155250515
70s의 기각역 : 0.012379518029708327


# 각연령대의 기각역을 초과하는 비율이 나온다면 표본 데이터셋이 전체 데이터셋을 대표하지 못하므로 가정에 어긋난다.

In [449]:
print('0s의 추정비율 :', (s0+(0.6985*np.sqrt(s0*(1-s0)/714)))*891)
print('1s의 추정비율 :', (s1+(0.6985*np.sqrt(s1*(1-s1)/714)))*891)
print('2s의 추정비율 :', (s2+(0.6985*np.sqrt(s2*(1-s2)/714)))*891)
print('3s의 추정비율 :', (s3+(0.6985*np.sqrt(s3*(1-s3)/714)))*891)
print('4s의 추정비율 :', (s4+(0.6985*np.sqrt(s4*(1-s4)/714)))*891)
print('5s의 추정비율 :', (s5+(0.6985*np.sqrt(s5*(1-s5)/714)))*891)
print('6s의 추정비율 :', (s6+(2.101*np.sqrt(s6*(1-s6)/714)))*891)
print('7s의 추정비율 :', (s7+(2.447*np.sqrt(s7*(1-s7)/714)))*891)

0s의 추정비율 : 83.92842785958291
1s의 추정비율 : 135.4359955007888
2s의 추정비율 : 285.2918542390537
3s의 추정비율 : 218.25851900634393
4s의 추정비율 : 118.75666428593495
5s의 추정비율 : 65.73165674462423
6s의 추정비율 : 34.98532005341691
7s의 추정비율 : 16.774683846173943


In [450]:
0.09419576639683828+0.15200448428820293+0.32019287793384255+0.2449590561238428+0.13328469616827715+0.07377290319261978+0.03926523013851505+0.018826805663494886

1.0765018199056333

In [451]:
83.92842785958291 + 135.4359955007888 + 285.2918542390537 +218.25851900634393 + 118.75666428593495 +65.73165674462423+34.98532005341691+16.774683846173943

959.1631215359193

In [452]:
print('표본의 오차율 :', str((83.92842785958291 + 135.4359955007888 + 285.2918542390537 +218.25851900634393 + 118.75666428593495 +65.73165674462423+34.98532005341691+16.774683846173943-891)/891*100)+'%')

표본의 오차율 : 7.650181990563339%


In [453]:
#베이즈 공식을 위한 자료정리
base=sample['Age'].groupby([sample['Survived'],sample['Sex'],sample['Pclass'],sample['SibSp'], sample['Parch']]).value_counts()
base=pd.DataFrame([base])
base=base.T
base.columns=['probability']
base

probability
Survived Sex    Pclass SibSp Parch Age             
0        female 1      0     0     5.0            1
                       1     2     0.0            1
                                   2.0            1
                2      0     0     2.0            1
                                   3.0            1
...                                             ...
1        male   3      1     0     1.0            1
                             1     0.0            3
                                   2.0            1
                             2     0.0            1
                       4     2     0.0            1

[235 rows x 1 columns]

In [454]:
base.to_csv('./calculation.csv')

In [455]:
base1=pd.read_csv('./calculation.csv')
base1=base1['probability'].groupby([base1['Survived'], base1['Sex'], base1['Pclass'],base1['SibSp'],base1['Parch']]).sum()
base1=pd.DataFrame([base1])
base1=base1.T
base1

probability
Survived Sex    Pclass SibSp Parch             
0        female 1      0     0                1
                       1     2                2
                2      0     0                3
                       1     0                2
                             1                1
...                                         ...
1        male   3      0     2                1
                       1     0                3
                             1                4
                             2                1
                       4     2                1

[101 rows x 1 columns]

In [456]:
# 베이즈통계 구하기
for ind1,sum1 in zip(base1.index, base1['probability']):
  for ind2,val in zip(base.index, base['probability'].values):
    if ind1 == ind2[:5]:
      base['probability'].loc[ind2] = val/sum1
base

probability
Survived Sex    Pclass SibSp Parch Age             
0        female 1      0     0     5.0     1.000000
                       1     2     0.0     0.500000
                                   2.0     0.500000
                2      0     0     2.0     0.333333
                                   3.0     0.333333
...                                             ...
1        male   3      1     0     1.0     0.333333
                             1     0.0     0.750000
                                   2.0     0.250000
                             2     0.0     1.000000
                       4     2     0.0     1.000000

[235 rows x 1 columns]

In [457]:
base.to_csv('./bayesian.csv')

In [458]:
# 예컨대 살아남지못하고, 여성이면서 Pclass가 1인 사람은 10세 이전 혹은 20대 혹은 50대일 확률이 각각 1/3이라는것이다.

#이 자료를 토대로 원래 데이터셋을 채워보면,
base=pd.read_csv('./bayesian.csv')
base

,Survived,Sex,Pclass,SibSp,Parch,Age,probability
0,0,female,1,0,0,5.0,1.000000
1,0,female,1,1,2,0.0,0.500000
2,0,female,1,1,2,2.0,0.500000
3,0,female,2,0,0,2.0,0.333333
4,0,female,2,0,0,3.0,0.333333
...,...,...,...,...,...,...,...
230,1,male,3,1,0,1.0,0.333333
231,1,male,3,1,1,0.0,0.750000
232,1,male,3,1,1,2.0,0.250000
233,1,male,3,1,2,0.0,1.000000


In [459]:
# base 데이터에서 더 높은 확률만을 갖는 데이터를 살아남게 한다.
# 곂치는 확률이 많으니 가중치를 구한다.
# 가중치 = probability * age
for Sur,Se,Pc,Pro,Sib,Par,inin in zip(base['Survived'], base['Sex'], base['Pclass'], base['probability'], base['SibSp'],base['Parch'],base.index):
  for Sur1,Se1,Pc1,Pro1,Sib1,Par1, inin1 in zip(base['Survived'], base['Sex'], base['Pclass'], base['probability'], base['SibSp'],base['Parch'], base.index):
    if Sur == Sur1 and Se == Se1 and Pc == Pc1 and Sib == Sib1 and Par == Par1 and Pro > Pro1:
      base['probability'].loc[inin1]=np.nan
base=base.dropna()
base['weighted_age']=base['Age']*base['probability']
base=base['weighted_age'].groupby([base['Survived'],base['Sex'],base['Pclass'],base['SibSp'],base['Parch']]).mean()
base=pd.DataFrame([base])
base=base.T
base.to_csv('./weighted_age.csv')

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [460]:
base=pd.read_csv('./weighted_age.csv')
base

,Survived,Sex,Pclass,SibSp,Parch,weighted_age
0,0,female,1,0,0,5.000000
1,0,female,1,1,2,0.500000
2,0,female,2,0,0,1.111111
3,0,female,2,1,0,1.500000
4,0,female,2,1,1,2.000000
...,...,...,...,...,...,...
96,1,male,3,0,2,0.000000
97,1,male,3,1,0,1.333333
98,1,male,3,1,1,0.000000
99,1,male,3,1,2,0.000000


In [461]:
#원본 데이터도 나이대를 바꾸어주기

for older1,ind11 in zip(data['Age'],data.index):
  if older1 <10 :
    data['Age'].loc[ind11]=0 
  elif older1 <20:
    data['Age'].loc[ind11]=1
  elif older1 <30:
    data['Age'].loc[ind11]=2
  elif older1 <40:
    data['Age'].loc[ind11]=3
  elif older1 <50:
    data['Age'].loc[ind11]=4
  elif older1 <60:
    data['Age'].loc[ind11]=5
  elif older1 <70:
    data['Age'].loc[ind11]=6
  elif older1 <=90:
    data['Age'].loc[ind11]=7
data

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,3,male,2.0,1,0,A/5 21171,7.2500,NaN,S
1,1,1,female,3.0,1,0,PC 17599,71.2833,C85,C
2,1,3,female,2.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,1,female,3.0,1,0,113803,53.1000,C123,S
4,0,3,male,3.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,2.0,0,0,211536,13.0000,NaN,S
887,1,1,female,1.0,0,0,112053,30.0000,B42,S
888,0,3,female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,1,1,male,2.0,0,0,111369,30.0000,C148,C


# 구해놓은 베이즈 통계를 이용하여 결측값을 채운다.

In [466]:
data=data.fillna('Na')
for Sur,Se,Pc,Sib,Par,ag,inin in zip(base['Survived'], base['Sex'], base['Pclass'],base['SibSp'],base['Parch'],base['weighted_age'],base.index):
  for Sur1,Se1,Pc1,Sib1,Par1,agag, inin1 in zip(data['Survived'], data['Sex'], data['Pclass'],data['SibSp'],data['Parch'],data['Age'], data.index):
    if Sur == Sur1 and Se == Se1 and Pc == Pc1 and Sib == Sib1 and Par == Par1 and agag == str('Na'):
      data['Age'].loc[inin1]=ag


In [467]:
data

,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,3,male,2,1,0,A/5 21171,7.2500,Na,S
1,1,1,female,3,1,0,PC 17599,71.2833,C85,C
2,1,3,female,2,0,0,STON/O2. 3101282,7.9250,Na,S
3,1,1,female,3,1,0,113803,53.1000,C123,S
4,0,3,male,3,0,0,373450,8.0500,Na,S
...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,2,0,0,211536,13.0000,Na,S
887,1,1,female,1,0,0,112053,30.0000,B42,S
888,0,3,female,Na,1,2,W./C. 6607,23.4500,Na,S
889,1,1,male,2,0,0,111369,30.0000,C148,C
